<a href="https://colab.research.google.com/github/Indirapriyadarshini/Document-Summerization/blob/main/Copy_of_Flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/template')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install flask-ngrok

In [ ]:
!pip install sumy
import sumy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask,render_template,url_for,request
import time
import spacy
import nltk
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from bs4 import BeautifulSoup
from urllib.request import urlopen,Request

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
app = Flask(__name__)

def lex_summary(docx):
	parser = PlaintextParser.from_string(docx,Tokenizer("english"))
	lex_summarizer = LexRankSummarizer()
	summary = lex_summarizer(parser.document,3)
	summary_list = [str(sentence) for sentence in summary]
	result = ' '.join(summary_list)
	return result

def luhn_summary(docx):
	parser = PlaintextParser.from_string(docx,Tokenizer("english"))
	summarizer_luhn = LuhnSummarizer()
	summary_1 =summarizer_luhn(parser.document,3)
	summary_list = [str(sentence) for sentence in summary_1]
	result = ' '.join(summary_list)
	return result


def isa_summary(docx):
	parser = PlaintextParser.from_string(docx,Tokenizer("english"))
	summarizer_lsa = LsaSummarizer()
	summary_2 =summarizer_lsa(parser.document,3)
	summary_list = [str(sentence) for sentence in summary_2]
	result = ' '.join(summary_list)
	return result


In [ ]:
# Reading Time
def readingTime(mytext):
	total_words = len([ token.text for token in nlp(mytext)])
	estimatedTime = total_words/200.0
	return estimatedTime

In [ ]:
@app.route('/')
def index():
	return render_template('index.html')

@app.route('/process',methods=['GET','POST'])
def process():
    start = time.time()
    if request.method == 'POST':
        input_text = request.form['input_text']
        model_choice = request.form['model_choice']
        final_reading_time = readingTime(input_text)
        if model_choice == 'default':
            final_summary = lex_summary(input_text)
        elif model_choice == 'lex_summarizer':
            final_summary = lex_summary(input_text)
        elif model_choice == 'luhn_summarizer':
            final_summary= luhn_summary(input_text)
        elif model_choice == 'isa_summarizer':
            final_summary= isa_summary(input_text)
    summary_reading_time = readingTime(final_summary)
    end = time.time()
    final_time = end-start
    return render_template('summary.html',ctext=input_text,final_reading_time=final_reading_time,summary_reading_time=summary_reading_time,final_summary=final_summary,model_selected=model_choice)

In [ ]:
from bs4 import BeautifulSoup
# from urllib.request import urlopen
from urllib.request import urlopen

In [ ]:
def get_text(url):
    reqt = Request(url,headers={'User-Agent' : "Magic Browser"})
    page = urlopen(reqt)
    soup = BeautifulSoup(page)
    fetched_text = ' '.join(map(lambda p:p.text,soup.find_all('p')))
    return fetched_text

In [ ]:
@app.route('/process_url',methods=['GET','POST'])
def process_url():
	start = time.time()
	if request.method == 'POST':
		input_url = request.form['input_url']
		raw_text = get_text(input_url)
		final_reading_time = readingTime(raw_text)
		final_summary = lex_summary(raw_text)
		summary_reading_time = readingTime(final_summary)
		end = time.time()
		final_time = end-start
	return render_template('summary.html',ctext=raw_text,
                        final_summary=final_summary,
                        final_time=final_time,
                        final_reading_time=final_reading_time,
                        summary_reading_time=summary_reading_time)

In [ ]:
if __name__ == '__main__':
	app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


**Mounting Google Colab Terminal**

In [ ]:
from IPython.display import JSON
from google.colab import output
from subprocess import getoutput
import os
def shell(command):
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])
  return JSON([getoutput(command)])
output.register_callback('shell', shell)

In [ ]:
#@title Colab Shell
%%html
<div id=term_demo></div>
<script src="https://code.jquery.com/jquery-latest.js"></script>
<script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
<link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
<script>
  $('#term_demo').terminal(async function(command) {
      if (command !== '') {
          try {
              let res = await google.colab.kernel.invokeFunction('shell', [command])
              let out = res.data['application/json'][0]
              this.echo(new String(out))
          } catch(e) {
              this.error(new String(e));
          }
      } else {
          this.echo('');
      }
  }, {
      greetings: 'Welcome to Colab Shell',
      name: 'colab_demo',
      height: 250,
      prompt: 'colab > '
  });

In [ ]:
!pip install scrubadub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scrubadub-2.0.0-py3-none-any.whl (65 kB)
  Using cached textblob-0.15.3-py2.py3-none-any.whl (636 kB)
  Using cached phonenumbers-8.13.13-py2.py3-none-any.whl (2.6 MB)
  Using cached python_stdnum-1.18-py2.py3-none-any.whl (1.0 MB)
  Using cached dateparser-1.1.8-py2.py3-none-any.whl (293 kB)
  Using cached sklearn-0.0.post5.tar.gz (3.7 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.7 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2950 sha256=fd25c6d526690bedb66f510f08dac3acdb98be019b7457564f9c7aef01ca476c
  Stored in directory: /root/.cache/pip/wheels/38/1f/8d/4f812c590e074c1e928f5cec67bf5053b71f38e2648739403a
Successfully built sklearn
  Attempting uninstall: textblob
    Found existing installation: textblob 0.17.1
    Uninstalling textblob-0.17.1:
      Successfully

In [ ]:
!pip install scrubadub_spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scrubadub_spacy-2.0.0-py3-none-any.whl (15 kB)


In [ ]:
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Remove Personal Identifiable Information**

In [ ]:
import scrubadub
text = "My name is Alex, I work at LifeGuard in London, and my eMail is alex@lifeguard.com btw. my super secret twitter login is username: alex_2000 password: g-dragon180888"

scrubadub.clean(text)

'My name is Alex, I work at LifeGuard in London, and my eMail is {{EMAIL}} btw. my super secret twitter login is username: {{USERNAME}} password: {{PASSWORD}}'

In [ ]:
!pip install scrubadub_stanford

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached scrubadub_stanford-2.1.3-py3-none-any.whl (16 kB)
  Using cached stanza-1.5.0-py3-none-any.whl (802 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.7/353.7 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.4.0-py2.py3-none-any.whl size=350809 sha256=71d9b4c81e289701dc43c31d9c85204b96ea68fc0afbbaa983ad41529646a535
  Stored in directory: /root/.cache/pip/wheels/79/29/1c/234cae4632803c2ba4a76a71a679eb1383cf590775714e2a21
Successfully built emoji


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import scrubadub, scrubadub_stanford
scrubber = scrubadub.Scrubber()
scrubber.add_detector(scrubadub_stanford.detectors.StanfordEntityDetector(
    enable_person=True, enable_organization=True, enable_location=True
))
scrubber.clean("My name is John and I work at the United Nations in Geneva")

'My name is {{NAME}} and I work at the {{ORGANIZATION}} in {{LOCATION}}'

https://www.decodeschool.com/blog/Text-Mining/Centroid-based-Text-summarization-in-Python

In [ ]:

from os import listdir
import string
import math

"""Method to calculate Inverse Document Frequency Score"""
def calculate_idf(word):
  files = [f for f in listdir("/content/drive/MyDrive/Hackathon/article") ]  #Specify the directory where the documents located
  count,wcount=2,1
  for file1 in files:
    file=open("/content/drive/MyDrive/Hackathon/article" +file1,'r')     #Specify the directory where the documents located
    page=file.read()
    if(word in page):
      wcount+=1
      count+=1
    idf=count/wcount

    return math.log(idf,10)

"""Method to calculate Centroid Score of sentences"""
def calculate_centroid(sentences):

    """"Compute tf X idf score for each word"""
    tfidf=dict()
    for sentence in sentences:
        words=sentence.split()
        for word in words:
            if word in tfidf:
                tfidf[word]+=calculate_idf(word)
            else:
                tfidf[word]=calculate_idf(word)

    """Construct the centroid of Cluster
    By taking the words that are above the threshold"""

    centroid=dict()
    threshold=0.7
    for word in tfidf:
        if(tfidf[word]>threshold):
            centroid[word]=tfidf[word]
        else:
            centroid[word]=0

    """Compute the Score for Sentences"""
    senctence_score=list()
    counter=0
    for sentence in sentences:
        senctence_score.append(0)
        words=sentence.split()
        for word in words:
            senctence_score[counter]+=centroid[word]

        counter=counter+1
    return senctence_score


"""Splitting Documents as sentences"""
files = [f for f in listdir("/content/drive/MyDrive/Hackathon/article") ]
page=""
for file1 in files:
  file = open("//content/drive/MyDrive/Hackathon/article" +file1,'r')
  page+=file.read()
  file.close()


sentences=page.split(".")
senctence_score=calculate_centroid(sentences)


"""Printing Sentences which has more central words"""
for i in range(len(sentences)):
  if(senctence_score[i]>15):
    print(sentences[i])




FileNotFoundError: ignored

**Multi-document Summarization**

In [ ]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 72.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=c8d07bfd5c82a7d1645879b3f28025f6d64d1c54a44c7c7bb9d721a78bcef56c
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [ ]:
!pip install PyPDF2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.6 MB/s eta 0:00:00


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import docx
import PyPDF2


In [ ]:
def preprocess_text(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Remove stopwords and perform stemming
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    preprocessed_sentences = []
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        filtered_words = [stemmer.stem(word.lower()) for word in words if word.lower() not in stop_words]
        preprocessed_sentences.append(' '.join(filtered_words))

    return preprocessed_sentences

def read_text_from_docx(file_path):
    doc = docx.Document('/content/drive/MyDrive/Hackathon/article/Copy of Project Methodology.docx')
    text1 = ' '.join([paragraph.text for paragraph in doc.paragraphs])
    return text1
#text1 = read_text_from_docx('Copy of Project Methodology.docx')

def read_text_from_pdf(file_path):
    with open('/content/drive/MyDrive/Hackathon/article/Copy of roughness.pdf', 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        text2 = ''
        for page in range(num_pages):
            page_obj = reader.pages[page]
            text2 += page_obj.extract_text()
    return text2


#text2 = read_text_from_pdf('Copy of roughness.pdf')

def read_text_from_file(file_path):
   _, ext = os.path.splitext(file_path)
   if ext == '.docx':
    return read_text_from_docx(file_path)
   elif ext == '.pdf':
    return read_text_from_pdf(file_path)
   else:
    with open(file_path, 'r') as f:
       return f.read()

In [ ]:
def calculate_sentence_scores(sentences):
    # Calculate the frequency of each word in the sentences
    word_frequencies = {}
    for sentence in sentences:
        for word in nltk.word_tokenize(sentence):
            if word not in word_frequencies:
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    # Normalize the word frequencies
    maximum_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = word_frequencies[word] / maximum_frequency

    # Calculate the score of each sentence based on word frequencies
    sentence_scores = {}
    for sentence in sentences:
        for word in nltk.word_tokenize(sentence):
            if sentence not in sentence_scores:
                sentence_scores[sentence] = word_frequencies[word]
            else:
                sentence_scores[sentence] += word_frequencies[word]

    return sentence_scores

def generate_summary(documents):
    # Preprocess the text in the documents
    preprocessed_documents = [preprocess_text(document) for document in documents]

    # Combine all the preprocessed sentences into a single list
    sentences = [sentence for document in preprocessed_documents for sentence in document]

    # Calculate the scores of the sentences
    sentence_scores = calculate_sentence_scores(sentences)

    # Sort the sentences based on their scores in descending order
    sorted_sentences = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)

    # Select the top n sentences as the summary
    summary_sentences = [sentence for sentence, score in sorted_sentences[:3]]  # Change 3 to the desired number of sentences

    # Combine the summary sentences into a single string
    summary = ' '.join(summary_sentences)

    return summary


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=37b611766412efe2eb6864bef54108d3d70956e985bd379a9931d84e75e7a9f1
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [ ]:
text1 = read_text_from_docx('Copy of Project Methodology.docx')
text2 = read_text_from_pdf('Copy of roughness.pdf')
# Example usage
document_files = ['text1.docx', 'text2.pdf']

documents = []
for file in document_files:
  document = read_text_from_file(file)
  documents.append(document)

summary = generate_summary(documents)
print(summary)

NameError: ignored